<a href="https://colab.research.google.com/github/jasleenkaursandhu/Reproducing-chest-xray-report-generation-boag/blob/main/3gram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# N-gram Model for Report Generation
# This notebook implements an n-gram language model for chest X-ray report generation.
# The model selects the most similar training images for each test image and builds a language model from their reports.


In [2]:
# docker image code to made files labelled for chexpert
# For 1-gram
##docker run --platform linux/amd64 -v /Users/jasleensandhu/Desktop/CS598DLH:/data uwizeye2/chexpert-labeler:amd64 python label.py --reports_path /data/1gram_headerless.csv --output_path /data/output/labeled_1gram.csv --verbose

# For 2-gram
##docker run --platform linux/amd64 -v /Users/jasleensandhu/Desktop/CS598DLH:/data uwizeye2/chexpert-labeler:amd64 python label.py --reports_path /data/2gram_headerless.csv --output_path /data/output/labeled_2gram.csv --verbose

# For 3-gram
##docker run --platform linux/amd64 -v /Users/jasleensandhu/Desktop/CS598DLH:/data uwizeye2/chexpert-labeler:amd64 python label.py --reports_path /data/3gram_headerless.csv --output_path /data/output/labeled_3gram.csv --verbose

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import tqdm
from collections import defaultdict, Counter
import pickle
import gzip
import random
import re
import warnings
!pip install pydicom
import pydicom
from time import gmtime, strftime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.6 MB/s eta 0:00:00


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/mimic-cxr-project'
!mkdir -p {base_path}/data
!mkdir -p {base_path}/output

Mounted at /content/drive


In [5]:
# Import the report parser module
import sys
sys.path.append(f"{base_path}/modules")
from report_parser import parse_report, MIMIC_RE
print("Successfully imported report parser module")

Successfully imported report parser module


In [6]:
# Load train and test data
data_dir = os.path.join(base_path, 'data')
files_path = os.path.join(base_path, 'files')
output_dir = os.path.join(base_path, 'output')
features_dir = os.path.join(base_path, 'features')

train_df = pd.read_csv(os.path.join(base_path, 'local_output/data/train.tsv'), sep='\t')
test_df = pd.read_csv(os.path.join(base_path, 'local_output/data/test.tsv'), sep='\t')

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

Train data shape: (4291, 3)
Test data shape: (1757, 3)


In [7]:
# Load the top 100 neighbors for each test image
neighbors_path = os.path.join(output_dir, 'top100_neighbors.pkl')

if os.path.exists(neighbors_path):
    with open(neighbors_path, 'rb') as f:
        neighbors = pickle.load(f)

    print(f"Loaded neighbors for {len(neighbors)} test images")
    print(f"Sample neighbors for first test image: {list(neighbors.items())[0][1][:5]}...")
else:
    print(f"Warning: Neighbors file not found at {neighbors_path}")
    print("Please run the KNN model first to generate the neighbors file.")
    neighbors = {}
    for dicom_id in test_df.dicom_id.values:
        neighbors[dicom_id] = random.sample(train_df.dicom_id.tolist(), min(100, len(train_df)))

Loaded neighbors for 868 test images
Sample neighbors for first test image: ['9368d351-90051a0c-c8dc80c0-96a2254d-2a884177', '6cad5cfd-bf8d1805-f3a3906a-588e0bfe-fe5c153d', 'd6c4c0d6-f12c2415-a5b3d2df-bcc4a202-442d3523', 'd18b8527-4c5a26fd-ac1041dd-8f9fd75f-4327f46f', '6083cad9-c1727fa2-b948d959-763ca27c-6391e63b']...


In [8]:
# Map each dicom to its study_id
report_id_column = 'study_id'
report_lookup = dict(train_df[['dicom_id', report_id_column]].values)
print(f"Created lookup dictionary for {len(report_lookup)} training images")

Created lookup dictionary for 4291 training images


In [9]:
# Define tokens for sequence boundaries
START = '<START>'
END = '<END>'

# Build n-gram language model from neighbors
def fit(dicom_ids, n=3):
    """Build language model from the reports of the given dicom_ids"""
    # Language model maps context (n-1 previous words) to possible next words
    LM = defaultdict(Counter)

    for dicom_id in dicom_ids:
        if dicom_id not in report_lookup:
            continue

        report_id = report_lookup[dicom_id]

        # Get corresponding subject_id
        subject_row = train_df[train_df.dicom_id == dicom_id]
        if len(subject_row) == 0:
            continue

        subject_id = subject_row.iloc[0]['subject_id']

        # Construct path to the report
        subject_prefix = f"p{str(subject_id)[:2]}"
        subject_dir = f"p{subject_id}"
        study_dir = f"s{report_id}"
        report_file = f"{study_dir}.txt"
        report_path = os.path.join(files_path, subject_prefix, subject_dir, report_file)

        # Parse the report
        try:
            if os.path.exists(report_path):
                parsed_report = parse_report(report_path)

                if 'findings' in parsed_report:
                    # Tokenize the findings text
                    toks = parsed_report['findings'].replace('.', ' . ').split()

                    # Add padding tokens at the beginning and END token at the end
                    padded_toks = [START for _ in range(n-1)] + toks + [END]

                    # Build n-gram model by counting follow words for each context
                    for i in range(len(padded_toks) - n + 1):
                        context = tuple(padded_toks[i:i+n-1])
                        target = padded_toks[i+n-1]
                        sim = 1
                        LM[context][target] += sim
        except Exception as e:
            continue

    return LM

In [10]:
# Sample from the n-gram model
def sample(LM, seq_so_far, n):
    """Sample the next word based on the n-gram language model"""
    last = tuple(seq_so_far[-(n-1):])

    if last not in LM or not LM[last]:
        # If context not found in model, return END token
        return END

    words, counts = zip(*LM[last].items())
    total = sum(counts)
    P = np.array(counts) / total

    # Sample next word based on probabilities
    choice = np.random.choice(words, p=P)
    return choice

In [11]:
# Set n-gram size
n = 3

# Generate reports for test images
generated_reports = {}

print(f"Generating reports with {n}-gram model...")
for pred_dicom in tqdm.tqdm(test_df.dicom_id.values):
    # Skip if we don't have neighbors for this test image
    if pred_dicom not in neighbors:
        print(f"Warning: No neighbors for {pred_dicom}")
        continue

    # Build n-gram model from the neighbors' reports
    nn = neighbors[pred_dicom]
    LM = fit(nn, n=n)

    # Skip if model is empty
    if not LM:
        print(f"Warning: Empty language model for {pred_dicom}")
        continue

    # Handle initialization differently for different n values
    if n == 1:
        # For 1-gram, we don't need context
        generated_toks = []
        current = START  # Start token won't actually be used for context
    else:
        # For n > 1, initialize with n-1 START tokens
        generated_toks = [START for _ in range(n-1)]
        current = generated_toks[-1]

    # Generate until END token or max length
    while current != END and len(generated_toks) < 100:
        next_word = sample(LM, generated_toks, n)
        generated_toks.append(next_word)
        current = next_word

    # Remove START tokens (if any) and potentially END token
    if n > 1:
        generated_toks = generated_toks[n-1:]
    if generated_toks and generated_toks[-1] == END:
        generated_toks = generated_toks[:-1]

    # Join tokens into text
    generated_text = ' '.join(generated_toks)
    generated_reports[pred_dicom] = generated_text

print(f"Generated reports for {len(generated_reports)} test images")

Generating reports with 3-gram model...


  0%|          | 0/1757 [00:00<?, ?it/s]

  0%|          | 5/1757 [06:00<23:43:04, 48.74s/it]

  1%|          | 14/1757 [06:22<5:17:35, 10.93s/it]

  1%|          | 18/1757 [06:49<4:37:45,  9.58s/it]

...

 91%|█████████ | 1597/1757 [14:15<00:02, 64.95it/s]

 93%|█████████▎| 1626/1757 [14:16<00:01, 66.53it/s]

 98%|█████████▊| 1723/1757 [14:16<00:00, 145.27it/s]

100%|██████████| 1757/1757 [14:16<00:00,  2.05it/s] 

Generated reports for 286 test images


In [12]:
# Save the generated reports
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

# Create output directory if it doesn't exist
pred_dir = os.path.join(base_path, 'output')
os.makedirs(pred_dir, exist_ok=True)

# Save the generated reports
pred_file = os.path.join(pred_dir, f'{n}-gram.tsv')
print(f"Saving predictions to {pred_file}")

with open(pred_file, 'w') as f:
    print('dicom_id\tgenerated', file=f)
    for dicom_id, generated in sorted(generated_reports.items()):
        # Clean up the text (remove any tabs)
        cleaned_text = generated.replace('\t', ' ')
        print(f'{dicom_id}\t{cleaned_text}', file=f)

print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2025-04-21 05:16:33
Saving predictions to /content/drive/MyDrive/mimic-cxr-project/output/3-gram.tsv
2025-04-21 05:16:34


In [13]:
# Display sample of generated reports
sample_count = min(3, len(generated_reports))
sample_dicoms = list(generated_reports.keys())[:sample_count]

for dicom_id in sample_dicoms:
    print(f"\nSample report for {dicom_id}:")
    report_text = generated_reports[dicom_id]

    # Print preview of the report (first 200 characters)
    if len(report_text) > 200:
        print(report_text[:200] + "...")
    else:
        print(report_text)


Sample report for 63100eab-9e8a8d90-392bc822-325de482-69a64e3b:
the patient in the appropriate clinical setting . there is bibasilar atelectasis is noted above the thoracic inlet which may reflect atelectasis but could be developing aspiration pneumonia . minimal ...

Sample report for 17269efa-b016a94d-1361e8df-ac428071-d1133672:
extensive right lower lobe . with the endotracheal tube tip courses below the field of view .

Sample report for 247d5e7b-66c77989-ca5fec41-608aaa71-eab4c699:
cardiomediastinal and hilar silhouettes are stable . no effusion
